In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

data = pd.read_csv('Popular_Baby_Names.csv')

In [2]:
data.iloc[0,:]

Year of Birth              2011
Gender                   FEMALE
Ethnicity              HISPANIC
Child's First Name    GERALDINE
Count                        13
Rank                         75
Name: 0, dtype: object

In [3]:
data

,Year of Birth,Gender,Ethnicity,Child's First Name,Count,Rank
0,2011,FEMALE,HISPANIC,GERALDINE,13,75
1,2011,FEMALE,HISPANIC,GIA,21,67
2,2011,FEMALE,HISPANIC,GIANNA,49,42
3,2011,FEMALE,HISPANIC,GISELLE,38,51
4,2011,FEMALE,HISPANIC,GRACE,36,53
...,...,...,...,...,...,...
57577,2014,MALE,WHITE NON HISPANIC,Yousef,18,94
57578,2014,MALE,WHITE NON HISPANIC,Youssef,24,88
57579,2014,MALE,WHITE NON HISPANIC,Yusuf,16,96
57580,2014,MALE,WHITE NON HISPANIC,Zachary,90,39


In [4]:
data['Gender'] = (data['Gender'] == 'FEMALE').astype(int)
data['Ethnicity'] = data['Ethnicity'].apply(lambda x: str(x).split()[0])

In [5]:
data = data.rename(columns = {"Child's First Name":"Name"})

In [6]:
Ethnicity = data.loc[:,'Ethnicity'].drop_duplicates()
Ethnicity

0      HISPANIC
206       WHITE
547       ASIAN
698       BLACK
Name: Ethnicity, dtype: object

In [7]:
for i in range(data.shape[0]):
    if str(data.loc[i,'Ethnicity']) == 'HISPANIC':
        data.loc[i,'Ethnicity'] = 0
    if str(data.loc[i,'Ethnicity']) == 'WHITE':
        data.loc[i,'Ethnicity'] = 1
    if str(data.loc[i,'Ethnicity']) == 'ASIAN':
        data.loc[i,'Ethnicity'] = 2
    else:
        data.loc[i,'Ethnicity'] = 3
data

,Year of Birth,Gender,Ethnicity,Name,Count,Rank
0,2011,1,3,GERALDINE,13,75
1,2011,1,3,GIA,21,67
2,2011,1,3,GIANNA,49,42
3,2011,1,3,GISELLE,38,51
4,2011,1,3,GRACE,36,53
...,...,...,...,...,...,...
57577,2014,0,3,Yousef,18,94
57578,2014,0,3,Youssef,24,88
57579,2014,0,3,Yusuf,16,96
57580,2014,0,3,Zachary,90,39


In [8]:
X = data.drop(columns = ['Name', 'Count', 'Rank'])
y = data[['Name', 'Count']]
X

,Year of Birth,Gender,Ethnicity
0,2011,1,3
1,2011,1,3
2,2011,1,3
3,2011,1,3
4,2011,1,3
...,...,...,...
57577,2014,0,3
57578,2014,0,3
57579,2014,0,3
57580,2014,0,3


In [9]:
y

,Name,Count
0,GERALDINE,13
1,GIA,21
2,GIANNA,49
3,GISELLE,38
4,GRACE,36
...,...,...
57577,Yousef,18
57578,Youssef,24
57579,Yusuf,16
57580,Zachary,90
